In [1]:
import os
gpu_idx= '3'
os.environ["CUDA_VISIBLE_DEVICES"]= gpu_idx
import torch
from tqdm import tqdm

from generative.networks.nets import DiffusionModelUNet, AutoencoderKL, SPADEDiffusionModelUNet

from config.model_config import defaultCFG

import yaml
import numpy as np
import nibabel as nib

from torch.cuda.amp import GradScaler, autocast
from generative.networks.schedulers import DDPMScheduler, DDIMScheduler
from monai import transforms

import glob

/opt/conda/envs/vcm/lib/python3.12/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/opt/conda/envs/vcm/lib/python3.12/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")
/opt/conda/envs/vcm/lib/python3.12/site-packages/generative/networks/layers/vector_quantizer.py:86: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)
/opt/conda/envs/vcm/lib/python3.12/site-packages/generative/network

In [2]:
class NiftiSaver:
    def __init__(self, output_dir: str) -> None:
        super().__init__()
        self.output_dir = output_dir
        self.affine = np.array(
            [
                [-1.0, 0.0, 0.0, 96.48149872],
                [0.0, 1.0, 0.0, -141.47715759],
                [0.0, 0.0, 1.0, -156.55375671],
                [0.0, 0.0, 0.0, 1.0],
            ]
        )
        
        self.set_output_dir(self.output_dir)
        
    def set_output_dir(self, output_dir):
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)
        

    def save(self, image_data: torch.Tensor, file_name: str, is_label=False) -> None:
        image_data = image_data.cpu().numpy()
        image_data = image_data[0, 0, ...]
        if is_label:
            image_data = image_data.astype(np.uint8)  
        else: 
            image_data = (image_data - image_data.min()) / (image_data.max() - image_data.min())
            image_data =  (image_data * 255).astype(np.uint8) 

        empty_header = nib.Nifti1Header()
        sample_nii = nib.Nifti1Image(image_data, self.affine, empty_header)
        nib.save(sample_nii, f"{str(self.output_dir)}/{file_name}.nii.gz")
        
    def save_label(self, img_path, file_name: str) -> None:
        img_nib = nib.load(img_path)
        empty_header = nib.Nifti1Header()
        sample_nii = nib.Nifti1Image(np.array(img_nib.dataobj), self.affine, empty_header)
        nib.save(sample_nii, f"{str(self.output_dir)}/{file_name}.nii.gz")

In [3]:
cfg = defaultCFG()

def load_yaml_to_dict(file_path):
    with open(file_path, 'r') as file:
        data = yaml.safe_load(file)
    return data

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('cuda setup')
print(f"\tUsing {device}: {gpu_idx}")

print('model construction, load the weights')
autoencoder = AutoencoderKL(**cfg.get_AE_CFG())
AE_weight_path = '/root/vcm/VCM/weights/autoencoder.pth'
autoencoder.load_state_dict(torch.load(AE_weight_path))
autoencoder.to(device)
print(f"\t AE done")

diffusion = DiffusionModelUNet(**cfg.get_DM_CFG())
Diff_weight_path = '/root/vcm/VCM/weights/diffusion_model.pth'
diffusion.load_state_dict(torch.load(Diff_weight_path))
diffusion.to(device)
print(f"\t Diffusion done")



/tmp/ipykernel_65987/3359844733.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  autoencoder.load_state_dict(torch.load(AE_weight_path))


cuda setup
	Using cuda: 3
model construction, load the weights
	 AE done


/tmp/ipykernel_65987/3359844733.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  diffusion.load_state_dict(torch.load(Diff_weight_path))


	 Diffusion done


In [21]:
# condition pairs 

age_min, age_max, age_mu, age_sd = 44, 82, 63.6, 7.5
lv_min, lv_max, lv_mu, lv_sd = 6995.68, 171375, 37968, 17770.2
brain_min, brain_max, brain_mu, brain_sd = 1144240, 1793910, 1485720, 73726.7

In [7]:
val_scheduler = DDIMScheduler(num_train_timesteps=1000, schedule="scaled_linear_beta", beta_start=0.0015, beta_end=0.0205, clip_sample=False)
val_scheduler.set_timesteps(num_inference_steps=200)

scale_factor = 1
print(scale_factor)

dst_dir = f'/root/sexDiff/samples'
saver = NiftiSaver(dst_dir)

sigma_multip = [-2., -1., 0., 1., 2.]

total_repeat = 30
for iteration in tqdm(range(total_repeat)):

    # cond.shape --> [1, 4]
    
    for age_coef in sigma_multip:
        for lv_coef in sigma_multip:
            for brain_coef in sigma_multip:
                # we define three cond values for age, LV and brain. 
                cond_label = f'Cond-A{age_coef}_LV{lv_coef}_brain{brain_coef}'
                os.makedirs(f'{dst_dir}/{cond_label}', exist_ok=True)
                
                age_val = age_mu + age_coef* age_sd
                lv_val = lv_mu + lv_coef* lv_sd
                brain_val = brain_mu + brain_coef* brain_sd
                
                age_val = (age_val - age_min) / (age_max - age_min)
                lv_val = (lv_val - lv_min) / (lv_max - lv_min)
                brain_val = (brain_val - brain_min) / (brain_max - brain_min)
                
                # sex-age-ventV-brainV
                cond_man = torch.tensor([0., age_val, lv_val, brain_val], device=device, dtype=torch.float32).view(1, 1, 4)
                cond_woman = torch.tensor([1., age_val, lv_val, brain_val], device=device, dtype=torch.float32).view(1, 1, 4)

                val_BZ = 1 
                noise = torch.randn((val_BZ, 3, 20, 28, 20))
                noise = noise.to(device)

                image_man = noise
                image_woman = noise.detach().clone()

                man_cond_concat = cond_man.view(val_BZ, 4, 1, 1, 1).to(device)
                man_cond_concat = man_cond_concat.expand(list(man_cond_concat.shape[0:2]) + list(image_man.shape[2:]))
                
                woman_cond_concat = cond_woman.view(val_BZ, 4, 1, 1, 1).to(device)
                woman_cond_concat = woman_cond_concat.expand(list(woman_cond_concat.shape[0:2]) + list(image_man.shape[2:]))

                with torch.no_grad():
                        progress_bar = val_scheduler.timesteps
                        for t in progress_bar:
                            # MAN sampling
                            timesteps = torch.Tensor((t,)).to(device).long()
                            ldm_out = diffusion(torch.cat((image_man, man_cond_concat), dim=1),
                                                    timesteps=timesteps,
                                                    context=cond_man,)
                            image_man, _ = val_scheduler.step(ldm_out, t, image_man)
                            
                            # WOMAN sampling
                            timesteps = torch.Tensor((t,)).to(device).long()
                            ldm_out = diffusion(torch.cat((image_woman, woman_cond_concat), dim=1),
                                                    timesteps=timesteps,
                                                    context=cond_woman,)
                            image_woman, _ = val_scheduler.step(ldm_out, t, image_woman)
                        
                        # decode to MRI
                        man_sample = autoencoder.decode_stage_2_outputs(image_man.to(device)/scale_factor)
                        woman_sample = autoencoder.decode_stage_2_outputs(image_woman.to(device)/scale_factor)
                saver.save(man_sample, f'{cond_label}/MAN-{iteration}')
                saver.save(woman_sample, f'{cond_label}/WOMAN-{iteration}')

1


100%|██████████| 30/30 [14:48:16<00:00, 1776.55s/it]  


In [ ]:
# run the following code sinpet or mp_synthseg.py file for performing segmentation with SynthSeg
!python -m /root/sexDiff/mp_synthseg.py

In [27]:

import os 
import glob
import shutil
from tqdm import tqdm 

In [24]:
dst_dir = '/root/sexDiff/seg_vol'

In [ ]:
csv_files = glob.glob('/root/sexDiff/samples/*/vol.csv')

for one in tqdm(csv_files):
    dir_path = os.path.dirname(one)
    post = dir_path.split('Cond-')[1]
    age, lv, brain = post.split('_')
    age_cond = age.split('A')[1]
    lv_cond = lv.split('LV')[1]
    brain_cond = brain.split('brain')[1]
    
    age_val = age_mu + float(age_cond) * age_sd
    lv_val = lv_mu + float(lv_cond) * lv_sd
    brain_val = brain_mu + float(brain_cond) * brain_sd
    
    shutil.copy(one, f'{dst_dir}/{age_val}-{lv_val}-{brain_val}.csv')

100%|██████████| 125/125 [00:00<00:00, 6620.13it/s]


In [37]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, shapiro, levene
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multitest import multipletests
import glob
import os

# 1. 모든 CSV 파일 목록 가져오기
file_list = glob.glob("/root/sexDiff/seg_vol/*.csv")  # 현재 디렉토리의 모든 .csv 파일
if not file_list:
    raise FileNotFoundError("No CSV files found in the current directory!")

# 2. 결과 저장용 리스트 초기화
case1_results_all = []  # 케이스 1 결과
case2_results_all = []  # 케이스 2 결과

# 3. 각 파일에 대해 분석 수행
for filename in tqdm(file_list):
    
    # 데이터 로드
    df = pd.read_csv(filename)
    filename = os.path.basename(filename)
    print(f"\nProcessing file: {filename}")
    
    # 입력 조건 추출 (파일명에서)
    age_input, lv_input, brain_vol_input = map(float, filename.replace(".csv", "").split("-"))
    
    # 성별 그룹 나누기
    df['Sex'] = df['subject'].str.split('-').str[0]
    male_df = df[df['Sex'] == 'MAN'].copy()
    female_df = df[df['Sex'] == 'WOMAN'].copy()
    
    # eTIV 정규화 함수
    def normalize_by_etiv(df, mean_etiv):
        normalized_df = df.copy()
        for col in df.columns:
            if col not in ['subject', 'Sex', 'total intracranial']:
                normalized_df[col] = df[col] * (mean_etiv / df['total intracranial'])
        return normalized_df
    
    # 전체 샘플의 eTIV 평균 계산
    mean_etiv = df['total intracranial'].mean()
    
    # 정규화된 데이터프레임 생성
    male_normalized = normalize_by_etiv(male_df, mean_etiv)
    female_normalized = normalize_by_etiv(female_df, mean_etiv)
    
    # 케이스 1: eTIV 정규화 후 성별 차이 분석
    atlas_columns = [col for col in df.columns if col not in ['subject', 'Sex', 'total intracranial']]
    case1_results = []
    for col in atlas_columns:
        male_values = male_normalized[col]
        female_values = female_normalized[col]
        
        # 정규성 및 등분산성 검정
        _, p_male_norm = shapiro(male_values)
        _, p_female_norm = shapiro(female_values)
        _, p_levene = levene(male_values, female_values)
        
        # t-검정
        t_stat, p_val = ttest_ind(male_values, female_values, equal_var=(p_levene > 0.05))
        
        # 효과 크기 (Cohen's d)
        mean_male = male_values.mean()
        mean_female = female_values.mean()
        pooled_std = np.sqrt(((len(male_values) - 1) * male_values.var() + (len(female_values) - 1) * female_values.var()) / (len(male_values) + len(female_values) - 2))
        cohen_d = (mean_male - mean_female) / pooled_std if pooled_std != 0 else 0
        
        case1_results.append({
            'Region': col,
            'Male Mean': mean_male,
            'Female Mean': mean_female,
            'T-statistic': t_stat,
            'P-value': p_val,
            'Cohen\'s d': cohen_d
        })
    
    # 결과 데이터프레임 생성 및 다중 비교 보정
    case1_df = pd.DataFrame(case1_results)
    case1_df['P-value (Bonferroni)'] = multipletests(case1_df['P-value'], alpha=0.05, method='bonferroni')[1]
    
    # 유의미한 결과 필터링
    significant_case1 = case1_df[case1_df['P-value (Bonferroni)'] < 0.05]
    num_significant = len(significant_case1)
    
    # 케이스 1 결과 요약 (유의미한 영역 중 상위 3개)
    case1_summary = {
        'File': filename,
        'Num Significant Regions': num_significant
    }
    if num_significant > 0:
        top_3 = significant_case1.nsmallest(3, 'P-value (Bonferroni)')[['Region', 'P-value (Bonferroni)', 'Cohen\'s d']]
        for i, row in top_3.iterrows():
            case1_summary[f'Top {i+1} Region'] = row['Region']
            case1_summary[f'Top {i+1} P-value'] = row['P-value (Bonferroni)']
            case1_summary[f'Top {i+1} Cohen\'s d'] = row['Cohen\'s d']
    
    case1_results_all.append(case1_summary)
    
    # 케이스 2: 정규화하지 않은 경우 - 입력 조건과 합성 경향성 비교
    ratios = {}
    for col in atlas_columns:
        ratios[col] = (df[col] / df['total intracranial']).mean()
    
    reference_values = {col: ratio * brain_vol_input for col, ratio in ratios.items()}
    reference_values['total intracranial'] = brain_vol_input
    # reference_values['left lateral ventricle'] = lv_input / 2
    # reference_values['right lateral ventricle'] = lv_input / 2
    
    tendency_results = []
    for col in ['total intracranial'] + atlas_columns:
        ref_value = reference_values[col]
        male_values = male_df[col]
        female_values = female_df[col]
        
        male_deviation = ((male_values - ref_value) / ref_value * 100).mean()
        female_deviation = ((female_values - ref_value) / ref_value * 100).mean()
        t_stat, p_val = ttest_ind(male_values, female_values)
        
        tendency_results.append({
            'Region': col,
            'Reference Value': ref_value,
            'Male Deviation (%)': male_deviation,
            'Female Deviation (%)': female_deviation,
            'T-statistic': t_stat,
            'P-value': p_val
        })
    
    tendency_df = pd.DataFrame(tendency_results)
    
    # 케이스 2 결과 요약 (편차 차이 상위 30개 영역)
    tendency_df['Deviation Difference'] = tendency_df['Male Deviation (%)'] - tendency_df['Female Deviation (%)']
    top_30_tendency = tendency_df.nlargest(30, 'Deviation Difference', keep='all')[['Region', 'Male Deviation (%)', 'Female Deviation (%)', 'P-value']]
    
    case2_summary = {'File': filename}
    for i, row in top_30_tendency.iterrows():
        case2_summary[f'Top {i+1} Tendency Region'] = row['Region']
        case2_summary[f'Top {i+1} Male Deviation (%)'] = row['Male Deviation (%)']
        case2_summary[f'Top {i+1} Female Deviation (%)'] = row['Female Deviation (%)']
        case2_summary[f'Top {i+1} P-value'] = row['P-value']
    
    case2_results_all.append(case2_summary)
    
    # 시각화 1: 상위 10개 영역 Bar Plot
    top_regions = tendency_df.nlargest(10, 'Deviation Difference', keep='all')['Region']
    plot_data = []
    for region in top_regions:
        for sex, deviation in [('Male', tendency_df[tendency_df['Region'] == region]['Male Deviation (%)'].iloc[0]),
                               ('Female', tendency_df[tendency_df['Region'] == region]['Female Deviation (%)'].iloc[0])]:
            plot_data.append({'Region': region, 'Sex': sex, 'Deviation (%)': deviation})
    
    plot_df = pd.DataFrame(plot_data)
    plt.figure(figsize=(12, 6))
    sns.barplot(x='Region', y='Deviation (%)', hue='Sex', data=plot_df)
    plt.title(f'Synthesis Tendency: Deviation from Reference Values (Top 10 Regions) - {filename}')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f"{filename.replace('.csv', '_barplot.png')}")
    plt.close()
    
    # 시각화 2: 전체 영역 Heatmap
    heatmap_data = tendency_df[['Region', 'Male Deviation (%)', 'Female Deviation (%)']].set_index('Region')
    heatmap_data.columns = ['Male', 'Female']
    plt.figure(figsize=(8, 12))
    sns.heatmap(heatmap_data, cmap='RdBu_r', center=0, annot=True, fmt='.2f', cbar_kws={'label': 'Deviation (%)'})
    plt.title(f'Synthesis Tendency Heatmap: Deviation from Reference Values - {filename}')
    plt.xlabel('Sex')
    plt.ylabel('Region')
    plt.tight_layout()
    plt.savefig(f"{filename.replace('.csv', '_heatmap.png')}")
    plt.close()

# 4. 결과 저장 - 케이스 1
case1_results_df = pd.DataFrame(case1_results_all)
case1_results_df.to_csv("case1_results.csv", index=False)
print("\nCase 1 results saved to 'case1_results.csv'")

# 5. 결과 저장 - 케이스 2
case2_results_df = pd.DataFrame(case2_results_all)
case2_results_df.to_csv("case2_results.csv", index=False)
print("Case 2 results saved to 'case2_results.csv'")

  0%|          | 0/125 [00:00<?, ?it/s]


Processing file: 78.6-37968.0-1559446.7.csv


  1%|          | 1/125 [00:00<01:17,  1.61it/s]


Processing file: 63.6-20197.8-1411993.3.csv


  2%|▏         | 2/125 [00:01<01:16,  1.61it/s]

...


Processing file: 48.6-20197.8-1411993.3.csv


 99%|█████████▉| 124/125 [01:22<00:00,  1.59it/s]


Processing file: 63.6-73508.4-1633173.4.csv


100%|██████████| 125/125 [01:23<00:00,  1.50it/s]


Case 1 results saved to 'case1_results.csv'
Case 2 results saved to 'case2_results.csv'
